In [1]:
## 引入必要的运行库
import folium
import pandas as pd
import json
import os

In [2]:
'''
分别将下载好的数据导入到DataFrame
数据来源：European Commission, Joint Research Centre (EC-JRC)/Netherlands Environmental Assessment Agency (PBL). Emissions Database for Global Atmospheric Research (EDGAR), release EDGAR v6.0_GHG (1970 - 2018) of May 2021. For the energy related sectors the activity data are mainly based on IEA data from IEA (2019) World Energy Balances, www.iea.org/statistics, All rights reserved, as modified by Joint Research Centre, European Commission.
下载地址：https://edgar.jrc.ec.europa.eu/dataset_ghg60#p1
'''
# 设置文件的地址
fileName_CO2_bio = os.getcwd()+'\\v60_GHG_CO2_org_short-cycle_C_1970_2018\\v60_CO2_org_short-cycle_C_1970_2018.xls'
fileName_CO2_Fossil = os.getcwd()+'\\v60_GHG_CO2_excl_short-cycle_org_C_1970_2018\\v60_CO2_excl_short-cycle_org_C_1970_2018.xls'
fileName_CH4= os.getcwd()+'\\v60_GHG_CH4_1970_2018\\v60_CH4_1970_2018.xls'
fileName_N2O = os.getcwd()+'\\v60_GHG_N2O_1970_2018\\v60_N2O_1970_2018.xls'

# 将数据导入到DATaFrame中
sheetName = 'TOTALS BY COUNTRY'
header = 9
CO2_FossilByCountry = pd.read_excel(fileName_CO2_Fossil, sheet_name = sheetName, header = header)
CO2_BioByCountry = pd.read_excel(fileName_CO2_bio, sheet_name = sheetName, header = header)
CH4_ByCountry = pd.read_excel(fileName_CH4, sheet_name = sheetName, header = header)
N2O_ByCountry = pd.read_excel(fileName_N2O, sheet_name = sheetName, header = header)
# N2O_ByCountry.head(10)

In [3]:
'''
数据清洗：
原则：只要存在缺失数据，删除所在行。
但是这次可视化2018年数据，所以只剔除2018年数据中有缺失值的行。
'''
# print(type(CO2_FossilByCountry))
CO2_FossilByCountry = CO2_FossilByCountry.dropna(axis=0, how='any', subset='Y_2018') 
# CO2_FossilByCountry.isnull().any()
# CO2_FossilByCountry.tail(20)
CO2_BioByCountry = CO2_BioByCountry.dropna(axis=0, how='any', subset='Y_2018')
CH4_ByCountry = CH4_ByCountry.dropna(axis=0, how='any', subset='Y_2018')
N2O_ByCountry = N2O_ByCountry.dropna(axis=0, how='any', subset='Y_2018')
# CO2_BioByCountry.isnull().any()

In [ ]:
'''
数据分类
CH4和N2O排放数据文件中包含了化石源和生物源
以fossil_bio关键字将化石和生物源CH4和N2O排放分别保存到不同的DataFrame
'''
# 新建空白的化石源和生物源的CH4和N2O的排放DataFrame，用于保存排放数据
CH4_FossilByCountry = pd.DataFrame(columns=list(CO2_FossilByCountry))
CH4_BioByCountry = pd.DataFrame(columns=list(CO2_FossilByCountry))
N2O_FossilByCountry = pd.DataFrame(columns=list(CO2_FossilByCountry))
N2O_BioByCountry = pd.DataFrame(columns=list(CO2_FossilByCountry))

# 遍历甲烷数据，依据fossil_bio字段，将化石源和生物源的CH4分别保存到CH4_FossilByCountry和CH4_BioByCountry中
for index, row in CH4_ByCountry.iterrows():
    if row['fossil_bio'] == 'bio':
        # print(type(row), row)
        CH4_BioByCountry = CH4_BioByCountry.append(row, ignore_index=True)
        # print(CH4_BioByCountry.shape)
    # if row['fossil_bio'] == 'fossil':
    else:
        # CH4_FossilByCountry = pd.concat([CH4_FossilByCountry, row], ignore_index=True)
        CH4_FossilByCountry = CH4_FossilByCountry.append(row, ignore_index=True)
        # print(CH4_FossilByCountry.shape)

# 同上
for index, row in N2O_ByCountry.iterrows():
    if row['fossil_bio'] == 'bio':
        # print(type(row), row)
        N2O_BioByCountry = N2O_BioByCountry.append(row, ignore_index=True)
        # print(CH4_BioByCountry.shape)
    # if row['fossil_bio'] == 'fossil':
    else:
        # CH4_FossilByCountry = pd.concat([CH4_FossilByCountry, row], ignore_index=True)
        N2O_FossilByCountry = N2O_FossilByCountry.append(row, ignore_index=True)
        # print(N2O_FossilByCountry.shape)

# print(CH4_BioByCountry.shape, CH4_FossilByCountry.shape, N2O_BioByCountry.shape, N2O_FossilByCountry.shape)
# CH4_BioByCountry.head(10)

In [5]:
'''
碳排放计算
全国碳排放=化石源CO2排放+生物源CO2排放+化石源CH4排放*化石源CH4GWP+生物源CH4排放*生物源CH4GWP+(化石源N2O排放+生物源N2O排放)*N2OGWP
'''

# 根据AR6设置GWP
GWP_CH4_fossil = 29.8
GWP_CH4_bio = 27
GWP_N2O = 273

# 以化石源CO2排放数据出现国家为基础，计算国家碳排放数据
TotalEmissionByCountry = pd.DataFrame() # 新建一个DataFrame用来保存数据
# CO2_FossilByCountry.head(10)
for index, row in CO2_FossilByCountry.iterrows():# 遍历化石源CO2排放数据
    countryName = row['Name'] # 以国家名为关键字，从其他DataFrame中找到对应数据
    CO2_Fossil = row['Y_2018'] # 读取化石源CO2排放数据
    # print(CO2_BioByCountry.loc[CO2_BioByCountry['Name'] == countryName]['Y_2018'])
    # print(countryName in CO2_BioByCountry['Name'].values)
    if any([countryName not in CO2_BioByCountry['Name'].values, countryName not in CH4_FossilByCountry['Name'].values, countryName not in CH4_BioByCountry['Name'].values, countryName not in N2O_FossilByCountry['Name'].values, countryName not in N2O_BioByCountry['Name'].values]):
        continue # 如果一个国家缺少任意一类数据，那就舍弃他
    
    # 在相应DataFrame中以国家名为关键字检索2018年排放数据并保存到对应变量中
    CO2_bio = CO2_BioByCountry.loc[CO2_BioByCountry['Name'] == countryName]['Y_2018'].values[0]
    # print(type(CO2_bio), CO2_bio)
    CH4_Fossil = CH4_FossilByCountry.loc[CH4_FossilByCountry['Name'] == countryName]['Y_2018'].values[0]
    # print('haha2')
    CH4_bio = CH4_BioByCountry.loc[CH4_BioByCountry['Name'] == countryName]['Y_2018'].values[0]
    # print('haha3')
    N2O_Fossil = N2O_FossilByCountry.loc[N2O_FossilByCountry['Name'] == countryName]['Y_2018'].values[0]
    # print('haha4')
    N2O_bio = N2O_BioByCountry.loc[N2O_BioByCountry['Name'] == countryName]['Y_2018'].values[0]
    # print('haha5')
    
    # 计算国家CO2、CH4和N2O排放并保存到变量中
    TotalEmission = CO2_Fossil + CO2_bio + CH4_Fossil * GWP_CH4_fossil + CH4_bio * GWP_CH4_bio + (N2O_Fossil + N2O_bio) * GWP_N2O
    # print(countryName,type(CO2_Fossil),CO2_Fossil, type(CO2_bio),CO2_bio, type(CH4_Fossil),CH4_Fossil,type(CH4_bio),CH4_bio,type(N2O_Fossil),N2O_Fossil,type(N2O_bio),N2O_bio, type(TotalEmission),TotalEmission)
    # print(type(row['Y_2018']), type(TotalEmission))

    # 以IPCC_annex，C_group_IM24_sh，Country_code_A3，Name，CO2_Fossil，CO2_bio，CH4_Fossil，CH4_bio，N2O_Fossil，N2O_bio，TotalEmission为关键字，建立一条DataFrame数据
    data = pd.DataFrame({'IPCC_annex':[row['IPCC_annex']], 'C_group_IM24_sh':[row['C_group_IM24_sh']], 'Country_code_A3':[row['Country_code_A3']], 'Name':[row['Name']],'CO2_Fossil':[CO2_Fossil], 'CO2_bio':[CO2_bio],'CH4_Fossil':[CH4_Fossil],'CH4_bio':[CH4_bio],'N2O_Fossil':[N2O_Fossil],'N2O_bio':[N2O_bio],'TotalEmission':[TotalEmission]})
    # print(type(data), data.shape)
    
    # 将新数据与原有数据合并，向数据集中添加数据
    TotalEmissionByCountry = pd.concat([TotalEmissionByCountry, data])
    # print(TotalEmissionByCountry)
    # print('hah')
    # break

# 冗余的检查，确保没有空值
# TotalEmissionByCountry = TotalEmissionByCountry.dropna()
# TotalEmissionByCountry.isnull().any()
# TotalEmissionByCountry.shape
# TotalEmissionByCountry.head(10)

In [6]:
'''
定义一个归一化函数df_norm：
输入：需要标准化的DataFrame，以及需要归一化的列名
输入：完成标准化的DataFrame，在原有DataFrame最后上增加一列归一化数据，
'''
def df_norm(df, *cols):
    df_n = df.copy()
    for col in cols:
        ma = df_n[col].max()
        mi = df_n[col].min()
        df_n[col + '_n'] = (df_n[col] - mi) / (ma - mi)
    return (df_n)
TotalEmissionByCountry_norm = df_norm(TotalEmissionByCountry,'TotalEmission')
# TotalEmissionByCountry_norm.head(10)

# import random
# for index, row in TotalEmissionByCountry_norm.iterrows():
#     TotalEmissionByCountry_norm.replace(row['TotalEmission_n'],random.uniform(1,100),inplace=True)
# TotalEmissionByCountry_norm.head(10)

In [7]:
'''
读取Geojson文件
'''
worldGeojsonPath = os.getcwd()+'\\allCountriesGeojson.json' # 设置Geojson文件位置
with open(worldGeojsonPath, 'r', encoding='UTF-8') as f: # 通过open()方法打开能够设置解码格式，便于处理Geojson中的中文字符
    worldGeojson = json.load(f) 

In [26]:
'''
定义一个设置地图基底的函数getMapObject：
输入：
1. baseSource：基底地图来源：0:OpenStreetMap, 1: 高德地图，2:腾讯地图，尽量不要用OpenStreetMap，他在藏南地区处理上有问题；
2. centerLoc：地图中心位置坐标，默认为[0E，0N]
3. baseLayerTitle:基底地图标题。
输出：
folium.Map的地图基底
'''
#
def getMapObject(baseSource=1, centerLoc=[0, 0], baseLayerTitle='baseLayer'):#0:OpenStreetMap, 1: 高德地图，2:腾讯地图
    if baseSource == 0:
        m = folium.Map(location=centerLoc,
                       min_zoom=0,
                       max_zoom=19,
                       zoom_start=2,
                       control=False,
                       control_scale=True
                       )

    elif baseSource == 1:
        #下面的程式将使用高德地图作为绘图的基底
        m = folium.Map(location=centerLoc,
                       zoom_start=5,
                       control_scale=True,
                       control=False,
                       tiles=None
                       )

        folium.TileLayer(tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=8&x={x}&y={y}&z={z}',
                         attr="&copy; <a href=http://ditu.amap.com/>高德地图</a>",
                         min_zoom=0,
                         max_zoom=19,
                         control=True,
                         show=True,
                         overlay=False,
                         name=baseLayerTitle
                         ).add_to(m)
    else:
        #下面的程式将使用腾讯地图作为绘图的基底
        m = folium.Map(location=centerLoc,
                       zoom_start=5,
                       control_scale=True,
                       control=False,
                       tiles=None
                       )

        folium.TileLayer(tiles='http://rt{s}.map.gtimg.com/realtimerender?z={z}&x={x}&y={y}&type=vector&style=0',
                         attr="&copy; <a href=http://map.qq.com/>腾讯地图</a>",
                         min_zoom=0,
                         max_zoom=19,
                         control=True,
                         show=True,
                         overlay=False,
                         name=baseLayerTitle
                         ).add_to(m)
    return m

world_map = getMapObject(1)
# world_map

In [ ]:
'''
使用排放数据创建Choropleth Map
'''

# def percent_name(total, percent):
#     bins = list()
#     total.sort(reverse=False)
#     for i in percent:
#         sumCount = 0
#         for n in total:
#             sumCount = sumCount + n
#             if sumCount > i * sum(total):
#                 bins.append(n)
#                 break
    
#     return bins

# bins = percent_name(TotalEmission, [0, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
# print(bins)
bins = list(TotalEmissionByCountry_norm["TotalEmission"].quantile([0, 0.55, 0.75, 0.75, 1])) # 排放总量的四个分位数为界设置图例


folium.Choropleth(     
    geo_data=worldGeojson,                       # 应用的Geojson文件
    data=TotalEmissionByCountry_norm,            # 排放数据
    columns=['Country_code_A3','TotalEmission'], # 数据和地理位置的联系，要展示的数据
    key_on='feature.properties.SOC',                         # geojson中的对应信息
    bins=bins,                                   # 图例
    fill_color='YlOrRd',
    fill_opacity=0.7,
    nan_fill_color='white',
    nan_fill_opacity=1,
    line_opacity=0.2,
    highlight=True,
    legend_name='碳排放量'    
).add_to(world_map)

world_map

In [30]:
world_map.save('Global_Emission_Map.html')